# 目的(問い40)

# フェーズ0：データ理解（5問）
1 テーブルには何行あるか？
2 カラム一覧を確認
3 NULLを含むカラムはどれか？
4 国は何カ国あるか？
5 最も古い／新しい取引日はいつか？
# フェーズ1：基本集計（10問）
6 総売上はいくらか？
7 注文数（InvoiceNo）は何件か？
8 ユニーク顧客数は？
9 国別の売上合計
10 国別の顧客数
11 商品（StockCode）数はいくつか？
12 最も売れた商品（数量）
13 最も売上が高い商品
14 平均注文金額はいくらか？
15 注文あたりの平均商品数
# フェーズ2：時間分析（10問）
16 月別売上推移
17 月別注文数
18 月別ユニーク顧客数
19 曜日別売上
20 時間帯別売上
21 月ごとの平均注文金額
22 前月比売上増減（Window）
23 売上が最大の月
24 注文数が最大の月
25 売上が急落した月はあるか？
# フェーズ3：顧客分析（9問）
26 上位10%顧客の売上構成比
27 最も頻繁に購入している顧客
28 平均購入回数（顧客単位）
29 1回しか購入していない顧客数
30 国×顧客数
31 国×平均注文金額
32 顧客の最初の購入日
33 顧客の最後の購入日
34 休眠顧客はどれくらいいるか？
# フェーズ4：商品分析（5問）
35 商品別売上ランキング
36 商品別購入顧客数
37 売上が特定顧客に依存している商品
38 月別に売れる商品は変わるか？
39 ロングテール構造になっているか？

In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("../../data/sql/db/sales.db")
df = pd.read_csv("../../data/sql/raw/online_retail.csv")

df.to_sql(
    "online_retail",
    conn,
    if_exists="replace",
    index=False
)
# df.to_sql(
#    name,      テーブル名（文字列）
#    con,       接続先（DB接続オブジェクト）
#    if_exists, 既存テーブルがあった場合の挙動
#    index      dfのindexを保存するか
#)

# online_retailテーブルとして sales.dbに格納
# ↓ SQLiteに書き込まれた行数

541909

In [ ]:
query = """
SELECT COUNT(*) FROM online_retail;
"""
# * 全行
# COUNT() = 行数を数える

# SQLは online_retailとは別に、新しい結果テーブルを作る

cursor = conn.cursor()
cursor.execute(query)
row_count = cursor.fetchone()[0]
# [0] → 1列目を取り出す
# 1行目の1列目の値を取り出す
# fetchone() 結果テーブルから次の1行を取る

# 全体像
# DB（SQLite）
#   ↑
# cursor ← SQLを実行する係を用意
#   ↑
# execute() ← (係への命令)SQLを送る
#   ↑
# fetchone() ← 結果を1行取る

print(row_count)
# 問い1への答え

541909


In [ ]:
query = """
PRAGMA table_info(online_retail);
"""
# PRAGMA table_info(指定のテーブル);
# テーブルのカラム一覧と属性を返す

cursor = conn.cursor()
cursor.execute(query)
columns_info = cursor.fetchall()
# 結果テーブルは毎回違う(結果テーブルはSQLごとに新しく生成)
# cursorは直前に実行したSQLの結果のみを持つ
# fetchall() 結果テーブルの残り全部を取る

for col in columns_info:
    print(col)
# タプルのリストからタプルを取り出し、printしていく

# (0, 'InvoiceNo', 'TEXT', 0, None, 0)

# 0  カラム番号 ０番目のカラム
# 'InvoiceNo'  カラム名 InvoiceNo
# 'TEXT'  データ型 文字列型
# 0 NOT  NULL 制約（1=あり / 0=なし）
# None  デフォルト値 デフォルト値なし
# 0   主キーかどうか（1=PK）

# ↓ 問い２への答え

(0, 'InvoiceNo', 'TEXT', 0, None, 0)
(1, 'StockCode', 'TEXT', 0, None, 0)
(2, 'Description', 'TEXT', 0, None, 0)
(3, 'Quantity', 'INTEGER', 0, None, 0)
(4, 'InvoiceDate', 'TEXT', 0, None, 0)
(5, 'UnitPrice', 'REAL', 0, None, 0)
(6, 'CustomerID', 'REAL', 0, None, 0)
(7, 'Country', 'TEXT', 0, None, 0)


In [ ]:
query = """
SELECT
    COUNT(*) AS total_rows,
    COUNT(InvoiceNo)    AS cnt_InvoiceNo,
    COUNT(StockCode)    AS cnt_StockCode,
    COUNT(Description)  AS cnt_Description,
    COUNT(Quantity)     AS cnt_Quantity,
    COUNT(InvoiceDate)  AS cnt_InvoiceDate,
    COUNT(UnitPrice)    AS cnt_UnitPrice,
    COUNT(CustomerID)   AS cnt_CustomerID,
    COUNT(Country)      AS cnt_Country
FROM online_retail;
"""
# SELECT　結果として表示したい列(計算結果)を書く
# AS 列名　結果テーブルの列名を決定

cursor = conn.cursor()
cursor.execute(query)
result = cursor.fetchone()
result

# (541909, 541909, 541909, 540455, 541909, 541909, 541909, 406829, 541909)
# よってDescriptionに1454,CustomerIDに135080欠損が ← 問い3への答え

(541909, 541909, 541909, 540455, 541909, 541909, 541909, 406829, 541909)

In [ ]:
query = """
SELECT
    COUNT(DISTINCT Country) AS cnt_countries
FROM online_retail;
"""
# DISTINCT  重複を除外

cursor = conn.cursor()
cursor.execute(query)
result = cursor.fetchone()
result
# ↓ 問い4への答え

(38,)

In [ ]:
query = """
SELECT
    MIN(InvoiceDate) AS oldest_date,
    MAX(InvoiceDate) AS newest_date
FROM online_retail;
"""
# MIN  最小値（最も古い日付）
# MAX  最大値（最も新しい日付）

cursor = conn.cursor()
cursor.execute(query)
result = cursor.fetchone()
result
# ↓ 問い5への答え

('2010-12-01 08:26:00', '2011-12-09 12:50:00')

In [ ]:
query = """
SELECT
    SUM(Quantity * UnitPrice) AS total_sales
FROM online_retail;
"""
# SUM(...)               全行の合計（総売上）
# Quantity * UnitPrice   量の列×値段の列

cursor = conn.cursor()
cursor.execute(query)
result = cursor.fetchone()
result
# ↓ 問い6への答え

(9747747.934,)

In [18]:
query = """
SELECT
    COUNT(DISTINCT InvoiceNo) AS cnt_invoices
FROM online_retail;
"""
# 問い4と同じコード 
# 重複を除外した時の列の種類をカウントするという本質が同じため

cursor = conn.cursor()
cursor.execute(query)
result = cursor.fetchone()
result
# ↓ 問い7への答え

(25900,)

In [ ]:
query = """
SELECT
    COUNT(DISTINCT CustomerID) AS cnt_customerid
FROM online_retail;
"""
# このCustomerIDには欠損が入っていた事に注意

cursor = conn.cursor()
cursor.execute(query)
result = cursor.fetchone()
result
# ↓ 問い8への答え 

(4372,)

In [ ]:
query = """
SELECT
    Country,
    SUM(Quantity * UnitPrice) AS total_sales
FROM online_retail
GROUP BY Country
ORDER BY total_sales DESC;
"""
# CountryをSELECTに書かないと、
#「どのグループの結果なのかを表す情報を出力できない」
# ため、CountryをSELECTに含めている

# GROUP BY Country  国ごとに行をまとめる
# ORDER BY ... DESC  指定の列の数値が大きい順に並べる

cursor = conn.cursor()
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)
# タプルのリストからタプルを取り出し、printしていく

# ↓ 問い9への答え（国別売上一覧）

('United Kingdom', 8187806.364)
('Netherlands', 284661.54)
('EIRE', 263276.82)
('Germany', 221698.21)
('France', 197403.9)
('Australia', 137077.27)
('Switzerland', 56385.35)
('Spain', 54774.58)
('Belgium', 40910.96)
('Sweden', 36595.91)
('Japan', 35340.62)
('Norway', 35163.46)
('Portugal', 29367.02)
('Finland', 22326.74)
('Channel Islands', 20086.29)
('Denmark', 18768.14)
('Italy', 16890.51)
('Cyprus', 12946.29)
('Austria', 10154.32)
('Hong Kong', 10117.04)
('Singapore', 9120.39)
('Israel', 7907.82)
('Poland', 7213.14)
('Unspecified', 4749.79)
('Greece', 4710.52)
('Iceland', 4310.0)
('Canada', 3666.38)
('Malta', 2505.47)
('United Arab Emirates', 1902.28)
('USA', 1730.92)
('Lebanon', 1693.88)
('Lithuania', 1661.06)
('European Community', 1291.75)
('Brazil', 1143.6)
('RSA', 1002.3100000000001)
('Czech Republic', 707.72)
('Bahrain', 548.4)
('Saudi Arabia', 131.17)


In [ ]:
query = """
SELECT
    Country,
    COUNT(DISTINCT CustomerID) AS cnt_customerid
FROM online_retail
GROUP BY Country
ORDER BY cnt_customerid DESC;
"""
# ;はSQL文の終了を意味する

cursor = conn.cursor()
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)
# ↓ 問い10への答え

('United Kingdom', 3950)
('Germany', 95)
('France', 87)
('Spain', 31)
('Belgium', 25)
('Switzerland', 21)
('Portugal', 19)
('Italy', 15)
('Finland', 12)
('Austria', 11)
('Norway', 10)
('Netherlands', 9)
('Denmark', 9)
('Channel Islands', 9)
('Australia', 9)
('Sweden', 8)
('Japan', 8)
('Cyprus', 8)
('Poland', 6)
('Unspecified', 4)
('USA', 4)
('Israel', 4)
('Greece', 4)
('Canada', 4)
('EIRE', 3)
('United Arab Emirates', 2)
('Malta', 2)
('Bahrain', 2)
('Singapore', 1)
('Saudi Arabia', 1)
('RSA', 1)
('Lithuania', 1)
('Lebanon', 1)
('Iceland', 1)
('European Community', 1)
('Czech Republic', 1)
('Brazil', 1)
('Hong Kong', 0)


In [ ]:
query = """
SELECT
    COUNT(DISTINCT StockCode) AS cnt_products
FROM online_retail;
"""
# 問い4と同じ

cursor = conn.cursor()
cursor.execute(query)
result = cursor.fetchone()
result
# ↓ 問い11への答え

(4070,)

In [ ]:
query = """
SELECT
    StockCode,
    SUM(Quantity) AS total_quantity
FROM online_retail
GROUP BY StockCode
ORDER BY total_quantity DESC
LIMIT 1;
"""
# LIMIT 数字 が初見
# 上から、指定の数の行のみを表示

cursor = conn.cursor()
cursor.execute(query)
result = cursor.fetchone()
result
# ↓ 問い12への答え(22197番の商品が56450個で最も売れた)

('22197', 56450)

In [ ]:
query = """
SELECT
    StockCode,
    SUM(Quantity * UnitPrice) AS total_sales
FROM online_retail
GROUP BY StockCode
ORDER BY total_sales DESC
LIMIT 1;
"""
cursor = conn.cursor()
cursor.execute(query)
result = cursor.fetchone()
result
#  問い13への答え(DOTという商品が売上206245.48でトップ)

('DOT', 206245.48)

In [ ]:
query = """
SELECT
    AVG(invoice_sales) AS avg_order_value
FROM (
    SELECT
        InvoiceNo,
        SUM(Quantity * UnitPrice) AS invoice_sales
    FROM online_retail
    GROUP BY InvoiceNo
);
"""
# サブクエリ  SQLの中のSQL 
# サブクエリ(今回は注文ごとの売上金額一覧)を使用し、平均を算出

# AVG(Quantity * UnitPrice)は　1行あたりの平均金額
# 一度の注文で複数種類商品の注文をすると、一度に2行以上になる事に注意

cursor = conn.cursor()
cursor.execute(query)
result = cursor.fetchone()
result
# ↓ 問い14への答え（平均注文金額）

(376.3609240926641,)

In [ ]:
query = """
SELECT
    AVG(invoice_sales) AS avg_order_quantity
FROM (
    SELECT
        InvoiceNo,
        SUM(Quantity) AS invoice_sales
    FROM online_retail
    GROUP BY InvoiceNo
);
"""
cursor = conn.cursor()
cursor.execute(query)
result = cursor.fetchone()
result
# ↓ 問い15への答え
# 数値が大きいが業者も含まれていると考えれば妥当

(199.86293436293437,)

In [ ]:
query ="""
SELECT
    strftime('%Y-%m', InvoiceDate) AS year_month,
    SUM(Quantity * UnitPrice) AS total_sales
FROM online_retail
GROUP BY year_month
ORDER BY year_month;
"""
# SELECT
#    列名,
#    … AS …
# FROM online_retail

# の列名がstrftime('%Y-%m', InvoiceDate) AS year_month,に
# InvoiceDateを元に、年(Y),月(m)のみで分別

# ASC(昇順) DESC(降順)　指定しないとASCの方に
cursor = conn.cursor()
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)
# ↓ 問い16への答え

('2010-12', 748957.02)
('2011-01', 560000.26)
('2011-02', 498062.65)
('2011-03', 683267.08)
('2011-04', 493207.121)
('2011-05', 723333.51)
('2011-06', 691123.12)
('2011-07', 681300.111)
('2011-08', 682680.51)
('2011-09', 1019687.622)
('2011-10', 1070704.67)
('2011-11', 1461756.25)
('2011-12', 433668.01)


In [29]:
query ="""
SELECT
    strftime('%Y-%m', InvoiceDate) AS year_month,
    COUNT(DISTINCT InvoiceNo) AS cnt_invoiceno
FROM online_retail
GROUP BY year_month
ORDER BY year_month;
"""
cursor = conn.cursor()
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)
# ↓ 問い17への答え

('2010-12', 2025)
('2011-01', 1476)
('2011-02', 1393)
('2011-03', 1983)
('2011-04', 1744)
('2011-05', 2162)
('2011-06', 2012)
('2011-07', 1927)
('2011-08', 1737)
('2011-09', 2327)
('2011-10', 2637)
('2011-11', 3462)
('2011-12', 1015)


In [30]:
query ="""
SELECT
    strftime('%Y-%m', InvoiceDate) AS year_month,
    COUNT(DISTINCT CustomerID) AS cnt_customerid
FROM online_retail
GROUP BY year_month
ORDER BY year_month;
"""
cursor = conn.cursor()
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)
# ↓ 問い18への答え

('2010-12', 948)
('2011-01', 783)
('2011-02', 798)
('2011-03', 1020)
('2011-04', 899)
('2011-05', 1079)
('2011-06', 1051)
('2011-07', 993)
('2011-08', 980)
('2011-09', 1302)
('2011-10', 1425)
('2011-11', 1711)
('2011-12', 686)


In [31]:
query = """
SELECT
    strftime('%w', InvoiceDate) AS weekday_num,
    SUM(Quantity * UnitPrice) AS total_sales
FROM online_retail
GROUP BY weekday_num
ORDER BY weekday_num;
"""
# %w 0～6で曜日を表す
cursor = conn.cursor()
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)
# ↓ 問い19への答え

('0', 805678.8910000001)
('1', 1588609.431)
('2', 1966182.791)
('3', 1734147.01)
('4', 2112519.0)
('5', 1540610.811)


In [ ]:
query = """
SELECT
    CASE
        WHEN CAST(strftime('%H', InvoiceDate) AS INTEGER) BETWEEN 0 AND 5
            THEN 'night'
        WHEN CAST(strftime('%H', InvoiceDate) AS INTEGER) BETWEEN 6 AND 11
            THEN 'morning'
        WHEN CAST(strftime('%H', InvoiceDate) AS INTEGER) BETWEEN 12 AND 17
            THEN 'afternoon'
        ELSE 'evening'
    END AS time_zone,
    SUM(Quantity * UnitPrice) AS total_sales
FROM online_retail
GROUP BY time_zone
ORDER BY total_sales DESC;
"""

# CASE
#    WHEN 条件A THEN ラベルA
#    WHEN 条件B THEN ラベルB
#    WHEN 条件C THEN ラベルC
#    ELSE ラベルD
# END AS 

# %H 分秒切り捨て

cursor = conn.cursor()
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)
# ↓ 問い20への答え（時間帯別売上）

('afternoon', 5989246.772)
('morning', 3555581.322)
('evening', 202919.84)


In [ ]:

query = """
SELECT
    year_month,
    AVG(invoice_sales) AS avg_order_value
FROM (
    SELECT
        InvoiceNo,
        strftime('%Y-%m', InvoiceDate) AS year_month,
        SUM(Quantity * UnitPrice) AS invoice_sales
    FROM online_retail
    GROUP BY InvoiceNo, year_month
)
GROUP BY year_month
ORDER BY year_month;
"""
# 月別平均注文　
# 複数行に広がる注文(InvoiceNo)を纏め、月のラベルも用意 → それを平均
cursor = conn.cursor()
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)
# ↓ 問い21への答え

('2010-12', 2025)
('2011-01', 1476)
('2011-02', 1393)
('2011-03', 1983)
('2011-04', 1744)
('2011-05', 2162)
('2011-06', 2012)
('2011-07', 1927)
('2011-08', 1737)
('2011-09', 2327)
('2011-10', 2637)
('2011-11', 3462)
('2011-12', 1015)


In [ ]:
query = """
WITH monthly_sales AS (
    SELECT
        strftime('%Y-%m', InvoiceDate) AS year_month,
        SUM(Quantity * UnitPrice) AS monthly_sales
    FROM online_retail
    GROUP BY year_month
)
SELECT
    year_month,
    monthly_sales,
    monthly_sales - LAG(monthly_sales) OVER (ORDER BY year_month) AS mom_change
FROM monthly_sales
ORDER BY year_month;
"""
# LAG() = 1行前の値を取る
# OVER (ORDER BY )  何順に並べたときの1行前か指定

# WITH … ASで前提を置いた上で、次のSELECTを実行

cursor = conn.cursor()
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)
# ↓ 問い22への答え

('2010-12', 748957.02, None)
('2011-01', 560000.26, -188956.76)
('2011-02', 498062.65, -61937.609999999986)
('2011-03', 683267.08, 185204.42999999993)
('2011-04', 493207.121, -190059.95899999997)
('2011-05', 723333.51, 230126.38900000002)
('2011-06', 691123.12, -32210.390000000014)
('2011-07', 681300.111, -9823.008999999962)
('2011-08', 682680.51, 1380.3989999999758)
('2011-09', 1019687.622, 337007.11199999996)
('2011-10', 1070704.67, 51017.04799999995)
('2011-11', 1461756.25, 391051.5800000001)
('2011-12', 433668.01, -1028088.24)


In [ ]:
query = """
SELECT
    strftime('%Y-%m', InvoiceDate) AS year_month,
    SUM(Quantity * UnitPrice) AS total_sales
FROM online_retail
GROUP BY year_month
ORDER BY total_sales DESC
LIMIT 1;
"""
cursor = conn.cursor()
cursor.execute(query)
result = cursor.fetchone()
result
# 問い23への答え

('2011-11', 1461756.25)

In [ ]:
query = """
SELECT
    strftime('%Y-%m', InvoiceDate) AS year_month,
    COUNT(DISTINCT InvoiceNo) AS cnt_invoice
FROM online_retail
GROUP BY year_month
ORDER BY cnt_invoice DESC
LIMIT 1;
"""
cursor = conn.cursor()
cursor.execute(query)
result = cursor.fetchone()
result
# 問い24への答え

('2011-11', 3462)

In [ ]:
query = """
WITH monthly_sales AS (
    SELECT
        strftime('%Y-%m', InvoiceDate) AS year_month,
        SUM(Quantity * UnitPrice) AS monthly_sales
    FROM online_retail
    GROUP BY year_month
),
monthly_diff AS (
    SELECT
        year_month,
        monthly_sales,
        monthly_sales - LAG(monthly_sales) OVER (ORDER BY year_month) AS mom_change
    FROM monthly_sales
)
SELECT
    year_month,
    monthly_sales,
    mom_change
FROM monthly_diff
WHERE mom_change < 0
ORDER BY mom_change ASC;
"""
# おおまかな全体像
# WITH 1 AS
# 2 AS
# SELECT
#    year_month,
#    1,
#    mom_change
# FROM 2

cursor = conn.cursor()
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)
# ↓ 問い25への答え
# 売り上げが下がった月の一覧　
# 今回のデータは年月の幅が狭いので一覧を目視し、急落した月を確認した方が良いと判断

('2011-12', 433668.01, -1028088.24)
('2011-04', 493207.121, -190059.95899999997)
('2011-01', 560000.26, -188956.76)
('2011-02', 498062.65, -61937.609999999986)
('2011-06', 691123.12, -32210.390000000014)
('2011-07', 681300.111, -9823.008999999962)


In [ ]:
query = """
WITH customer_sales AS (
    SELECT
        CustomerID,
        SUM(Quantity * UnitPrice) AS total_sales
    FROM online_retail
    WHERE CustomerID IS NOT NULL
    GROUP BY CustomerID
),
ranked_customers AS (
    SELECT
        CustomerID,
        total_sales,
        NTILE(10) OVER (ORDER BY total_sales DESC) AS sales_group
    FROM customer_sales
),
total_sales_all AS (
    SELECT
        SUM(total_sales) AS all_sales
    FROM customer_sales
)
SELECT
    SUM(r.total_sales) AS top10_sales,
    t.all_sales,
    SUM(r.total_sales) * 1.0 / t.all_sales AS top10_ratio
FROM ranked_customers r
CROSS JOIN total_sales_all t
WHERE r.sales_group = 1;
"""
# WHERE CustomerID IS NOT NULLは欠損のあるCUstomerID(顧客)を除外
# NTILE(x) 行数をx等分する

# FROM ranked_customers r
# CROSS JOIN total_sales_all t
# この部分でr,tという略称を定義

# Pythonは0スタートだが、SQLは1スタート

# FROM ranked_customers rとしているが、このままでは,ここに
# SUM(r.total_sales) * 1.0 / t.all_sales AS top10_ratio 
# のt.all_salesがテーブルに存在せず,計算できない 
# → CROSS JOIN total_sales_all t　でt.all_salesの内容もテーブルにくっつける

cursor = conn.cursor()
cursor.execute(query)
result = cursor.fetchone()
result
# ↓ 問い26への答え
# top10%顧客からの売り上げ　全体の売り上げ　比率(top10_sales ÷ total_sale_all)

(4984121.39, 8300065.814, 0.6004917914738838)

In [ ]:
query = """
SELECT
    CustomerID,
    COUNT(DISTINCT InvoiceDate) AS cnt_invoicedate
FROM online_retail
GROUP BY CustomerID
ORDER BY cnt_invoicedate DESC
LIMIT 2
"""
cursor = conn.cursor()
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)
# ↓ 問い27への答え
# Noneの次、2行目の14911.0が248回で最も注文数が多い(注文頻度が高い)顧客

(None, 3161)
(14911.0, 248)


In [ ]:
# 問い27別解
# WHEREでNoneの行を弾く
query = """
SELECT
    CustomerID,
    COUNT(DISTINCT InvoiceDate) AS cnt_invoicedate
FROM online_retail
WHERE CustomerID IS NOT NULL
GROUP BY CustomerID
ORDER BY cnt_invoicedate DESC
LIMIT 1;
"""
cursor = conn.cursor()
cursor.execute(query)
result = cursor.fetchone()
result

(14911.0, 248)

In [52]:
query = """
SELECT
    AVG(cnt_invoicedate) AS avg_order_number
FROM (
    SELECT
    CustomerID,
    COUNT(DISTINCT InvoiceDate) AS cnt_invoicedate
FROM online_retail
WHERE CustomerID IS NOT NULL
GROUP BY CustomerID
);
"""
cursor = conn.cursor()
cursor.execute(query)
result = cursor.fetchone()
result
# 問い14と同じコード
# ↓ 問い28への答え

(5.039798719121683,)

In [53]:
query = """
SELECT
    COUNT(*) AS one_time_customer_count
FROM (
    SELECT
        CustomerID,
        COUNT(DISTINCT InvoiceNo) AS cnt_invoice
    FROM online_retail
    WHERE CustomerID IS NOT NULL
    GROUP BY CustomerID
)
WHERE cnt_invoice = 1;
"""
cursor = conn.cursor()
cursor.execute(query)
result = cursor.fetchone()
result
# ↓ 問い29への答え

(1313,)

In [65]:
query = """
SELECT
    Country,
    COUNT(DISTINCT CustomerID) AS country_customer_count
FROM online_retail
WHERE CustomerID IS NOT NULL
GROUP BY Country
ORDER BY country_customer_count DESC;
"""
cursor = conn.cursor()
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)
# ↓ 問い30への答え

('United Kingdom', 3950)
('Germany', 95)
('France', 87)
('Spain', 31)
('Belgium', 25)
('Switzerland', 21)
('Portugal', 19)
('Italy', 15)
('Finland', 12)
('Austria', 11)
('Norway', 10)
('Netherlands', 9)
('Denmark', 9)
('Channel Islands', 9)
('Australia', 9)
('Sweden', 8)
('Japan', 8)
('Cyprus', 8)
('Poland', 6)
('Unspecified', 4)
('USA', 4)
('Israel', 4)
('Greece', 4)
('Canada', 4)
('EIRE', 3)
('United Arab Emirates', 2)
('Malta', 2)
('Bahrain', 2)
('Singapore', 1)
('Saudi Arabia', 1)
('RSA', 1)
('Lithuania', 1)
('Lebanon', 1)
('Iceland', 1)
('European Community', 1)
('Czech Republic', 1)
('Brazil', 1)


In [ ]:
query = """
SELECT
    Country,
    AVG(invoice_sales) AS avg_order_value
FROM (
    SELECT
        Country,
        InvoiceNo,
        SUM(Quantity * UnitPrice) AS invoice_sales
    FROM online_retail
    GROUP BY Country, InvoiceNo
)
GROUP BY Country
ORDER BY avg_order_value DESC;
"""
cursor = conn.cursor()
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)
# ↓ 問い31への答え

('Netherlands', 2818.431089108911)
('Australia', 1986.6271014492752)
('Lebanon', 1693.88)
('Japan', 1262.1650000000002)
('Brazil', 1143.6)
('RSA', 1002.3100000000001)
('Singapore', 912.039)
('Denmark', 893.7209523809523)
('Norway', 879.0865)
('Israel', 878.6466666666668)
('Sweden', 795.5632608695652)
('Greece', 785.0866666666666)
('Switzerland', 761.9641891891891)
('EIRE', 731.3245000000001)
('Hong Kong', 674.4693333333333)
('Cyprus', 647.3145)
('United Arab Emirates', 634.0933333333334)
('Iceland', 615.7142857142857)
('Canada', 611.0633333333334)
('Channel Islands', 608.6754545454546)
('Austria', 534.437894736842)
('Spain', 521.6626666666667)
('Finland', 465.1404166666667)
('France', 428.2080260303687)
('Lithuania', 415.265)
('Portugal', 413.62)
('Germany', 367.6587230514096)
('Unspecified', 365.36846153846153)
('United Kingdom', 348.50627240997704)
('Belgium', 343.78957983193277)
('Italy', 307.1001818181818)
('Poland', 300.5475)
('European Community', 258.35)
('Malta', 250.5469999999

In [71]:
query = """
SELECT
    CustomerID,
    MIN(InvoiceDate) AS first_purchase_date,
    MAX(InvoiceDate) AS last_purchase_date
FROM online_retail
WHERE CustomerID IS NOT NULL
GROUP BY CustomerID;
"""
cursor = conn.cursor()
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)
# ↓ 問い32,33への答え

(12346.0, '2011-01-18 10:01:00', '2011-01-18 10:17:00')
(12347.0, '2010-12-07 14:57:00', '2011-12-07 15:52:00')
(12348.0, '2010-12-16 19:09:00', '2011-09-25 13:13:00')
(12349.0, '2011-11-21 09:51:00', '2011-11-21 09:51:00')
(12350.0, '2011-02-02 16:01:00', '2011-02-02 16:01:00')
(12352.0, '2011-02-16 12:33:00', '2011-11-03 14:37:00')
(12353.0, '2011-05-19 17:47:00', '2011-05-19 17:47:00')
(12354.0, '2011-04-21 13:11:00', '2011-04-21 13:11:00')
(12355.0, '2011-05-09 13:49:00', '2011-05-09 13:49:00')
(12356.0, '2011-01-18 09:50:00', '2011-11-17 08:40:00')
(12357.0, '2011-11-06 16:07:00', '2011-11-06 16:07:00')
(12358.0, '2011-07-12 10:04:00', '2011-12-08 10:26:00')
(12359.0, '2011-01-12 12:43:00', '2011-12-02 11:21:00')
(12360.0, '2011-05-23 09:43:00', '2011-10-18 15:22:00')
(12361.0, '2011-02-25 13:51:00', '2011-02-25 13:51:00')
(12362.0, '2011-02-17 10:30:00', '2011-12-06 15:40:00')
(12363.0, '2011-04-11 13:29:00', '2011-08-22 10:18:00')
(12364.0, '2011-08-19 15:24:00', '2011-12-02 10:

In [ ]:
query = """
WITH last_date AS (
    SELECT
        MAX(InvoiceDate) AS max_date
    FROM online_retail
),
customer_last_purchase AS (
    SELECT
        CustomerID,
        MAX(InvoiceDate) AS last_purchase_date
    FROM online_retail
    WHERE CustomerID IS NOT NULL
    GROUP BY CustomerID
)
SELECT
    COUNT(*) AS dormant_customer_count
FROM customer_last_purchase c
CROSS JOIN last_date d
WHERE
    julianday(d.max_date) - julianday(c.last_purchase_date) >= 90;
"""
# last_date 基準日
# 90日以上,買い物していない客を休眠顧客と定義
# juliandayで日付を数値に変換し、差を取る

cursor = conn.cursor()
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)
# ↓ 問い34への答え

In [ ]:
query = """
SELECT
    StockCode,
    SUM(Quantity * UnitPrice) AS invoice_sales
FROM online_retail
GROUP BY StockCode
ORDER BY invoice_sales DESC
LIMIT 10;
"""
cursor = conn.cursor()
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)
#  ↓ 問い35への答え
# ランキングなので上から10行ぐらいが妥当と判断

('DOT', 206245.48)
('22423', 164762.19)
('47566', 98302.98)
('85123A', 97894.5)
('85099B', 92356.03)
('23084', 66756.59)
('POST', 66230.64)
('22086', 63791.94)
('84879', 58959.729999999996)
('79321', 53768.060000000005)


In [74]:
query = """
SELECT
    StockCode,
    COUNT(DISTINCT CustomerID) AS customer_no
FROM online_retail
GROUP BY StockCode
ORDER BY customer_no DESC
LIMIT 10;
"""
cursor = conn.cursor()
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)
# ↓ 問い36への答え

('22423', 887)
('85123A', 858)
('47566', 708)
('84879', 679)
('22720', 640)
('85099B', 636)
('21212', 636)
('22086', 615)
('22457', 590)
('22138', 585)


In [ ]:
query = """
WITH product_customer_sales AS (
    SELECT
        StockCode,
        CustomerID,
        SUM(Quantity * UnitPrice) AS customer_sales
    FROM online_retail
    WHERE CustomerID IS NOT NULL AND Quantity > 0
    GROUP BY StockCode, CustomerID
),
product_summary AS (
    SELECT
        StockCode,
        SUM(customer_sales) AS total_sales,
        MAX(customer_sales) AS max_customer_sales
    FROM product_customer_sales
    GROUP BY StockCode
)
SELECT
    StockCode,
    total_sales,
    max_customer_sales,
    max_customer_sales * 1.0 / total_sales AS dependency_ratio
FROM product_summary
ORDER BY dependency_ratio DESC
LIMIT 10;
"""
# AND Quantity > 0
# 返品がある商品の場合、依存率が100%を超える可能性も　それを除外

# 特定顧客に依存している商品を
# 最大顧客売上が占める割合(最大顧客売上 ÷ 商品総売上)が高い商品と定義

cursor = conn.cursor()
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)
# ↓ 問い37への答え

('16020C', 5.1, 5.1, 1.0)
('16161M', 10.5, 10.5, 1.0)
('16169N', 10.5, 10.5, 1.0)
('17001', 20.88, 20.88, 1.0)
('17028J', 2.52, 2.52, 1.0)
('17191A', 15.0, 15.0, 1.0)
('20667', 2.95, 2.95, 1.0)
('20678', 6.6, 6.6, 1.0)
('20703', 4.25, 4.25, 1.0)
('20816', 15.6, 15.6, 1.0)


In [ ]:
query = """
WITH monthly_product_sales AS (
    SELECT
        strftime('%Y-%m', InvoiceDate) AS year_month,
        StockCode,
        SUM(Quantity * UnitPrice) AS product_sales
    FROM online_retail
    WHERE CustomerID IS NOT NULL AND Quantity > 0
    GROUP BY year_month, StockCode
),
ranked_products AS (
    SELECT
        year_month,
        StockCode,
        product_sales,
        RANK() OVER (
            PARTITION BY year_month
            ORDER BY product_sales DESC
        ) AS sales_rank
    FROM monthly_product_sales
)
SELECT
    year_month,
    StockCode,
    product_sales,
    sales_rank
FROM ranked_products
WHERE sales_rank <= 5
ORDER BY year_month, sales_rank;
"""
# データ全体からPARTITION BYによってデータを区分し、そのデータ毎にランキングを作る
cursor = conn.cursor()
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)
# ↓ 問い38への答え

('2010-12', '22423', 17756.7, 1)
('2010-12', '85123A', 9602.05, 2)
('2010-12', '21623', 6938.49, 3)
('2010-12', '82484', 6685.23, 4)
('2010-12', '21137', 6248.82, 5)
('2011-01', '23166', 77183.6, 1)
('2011-01', '85123A', 14979.75, 2)
('2011-01', '22423', 10765.5, 3)
('2011-01', '21108', 6840.450000000001, 4)
('2011-01', '79321', 5747.57, 5)
('2011-02', '22423', 9559.65, 1)
('2011-02', '21623', 6810.39, 2)
('2011-02', '85099B', 5191.05, 3)
('2011-02', '85123A', 4912.65, 4)
('2011-02', '22693', 3799.11, 5)
('2011-03', '22423', 14797.4, 1)
('2011-03', '85099B', 8496.88, 2)
('2011-03', '47566', 6129.150000000001, 3)
('2011-03', 'POST', 5325.0, 4)
('2011-03', '85123A', 5262.5, 5)
('2011-04', '22423', 12721.5, 1)
('2011-04', 'M', 10108.5, 2)
('2011-04', '85123A', 9581.65, 3)
('2011-04', '47566', 7278.450000000001, 4)
('2011-04', '21108', 6610.8, 5)
('2011-05', '47566', 13437.95, 1)
('2011-05', 'POST', 13358.75, 2)
('2011-05', '22423', 11503.6, 3)
('2011-05', '85123A', 10700.42, 4)
('2011-05'

In [ ]:
query = """
WITH product_sales AS (
    SELECT
        StockCode,
        SUM(Quantity * UnitPrice) AS total_sales
    FROM online_retail
    WHERE Quantity > 0
    GROUP BY StockCode
),
ranked_products AS (
    SELECT
        StockCode,
        total_sales,
        SUM(total_sales) OVER () AS all_sales,
        SUM(total_sales) OVER (
            ORDER BY total_sales DESC
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS cumulative_sales
    FROM product_sales
)
SELECT
    StockCode,
    total_sales,
    cumulative_sales * 1.0 / all_sales AS cumulative_ratio
FROM ranked_products
ORDER BY total_sales DESC;
"""

# ROWS BETWEEN A AND B
# ウィンドウ関数で、どの行範囲を対象にするか指定する構文
# UNBOUNDED PRECEDING 上限なしで前にさかのぼる

cursor = conn.cursor()
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)
# ↓ 問い39への答え
# 上位10商品で精々1割 → 完全にロングテール構造

('DOT', 206248.77, 0.019375978131983402)
('22423', 174484.74, 0.03576789410125105)
('23843', 168469.6, 0.05159471956791439)
('85123A', 104518.8, 0.06141370652808462)
('47566', 99504.33, 0.0707616106252468)
('85099B', 94340.05, 0.07962435800439588)
('23166', 81700.92000000001, 0.08729972614978129)
('M', 78110.27, 0.09463777177014218)
('POST', 78101.88, 0.10197502919449819)
('23084', 66964.99, 0.10826603486618529)
('22086', 64952.29, 0.11436795804692593)
('84879', 59094.93, 0.11991961331929962)
('79321', 54117.76, 0.1250036898658503)
('22502', 51426.62, 0.12983494808145962)
('22197', 51354.02, 0.1346593859120922)
('23298', 43182.83, 0.13871618377690934)
('22386', 42436.24, 0.1427028434706549)
('23203', 42067.78, 0.146654888301473)
('21137', 40652.13, 0.1504739403224792)
('23284', 38166.64, 0.15405949373941005)
('22720', 38158.39, 0.15764427211259352)
('22960', 37128.88, 0.16113233348112938)
('20725', 36042.01, 0.16451828917721778)
('82484', 36030.22, 0.1679031372653327)
('22178', 33187.9